In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as sf
import pyspark.sql.types as st
from datetime import date

spark = SparkSession.builder.getOrCreate()

daily_country_confirmed_cases = "./data/Daily_Country_Wise_Confirmed_Cases.csv"

daily_df = spark.read.csv(daily_country_confirmed_cases,inferSchema=True,header=True)


In [10]:
# Change to importing this function
def date_check():
    today = date.today()
    return str(today)

2022-07-05


In [16]:
# Showing the change of cases over a month
daily_df.select('Country','2022-05-06', '2022-06-06').where(daily_df['2022-06-06'] >= 1).show(5)

+-------------+----------+----------+
|      Country|2022-05-06|2022-06-06|
+-------------+----------+----------+
|      England|         1|        73|
|     Portugal|         0|        10|
|        Spain|         0|        17|
|United States|         0|         1|
|        Italy|         0|         6|
+-------------+----------+----------+
only showing top 5 rows

